# RAG Exercise Solution

In [1]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [2]:
# We populated the RAG with the data from the data/calories.csv file in the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [3]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """
    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)


@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function to ask a questions specifically about malnutrition.

    Args:
        query: Question prompt
        max_results: The maximum number of results to return

    Returns:
        String response to the question (prompt)
    """

    results = nutrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Answer to your question:\n" + "\n".join(formatted_results)

In [4]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice
    You give concise answers based on the  two tools you have available.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool],
)

In [5]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are nutritious, pregnancy-friendly meal ideas with approximate calories per serving. Also include key nutrients to emphasize.

- Oatmeal bowl with iron-fortified cereal, milk, berries, and a sprinkle of nuts (about 350 kcal)
  - Key nutrients: iron, calcium, fiber, folate

- Greek yogurt parfait with berries, chia/flaxseed, and a handful of nuts (about 300 kcal)
  - Key nutrients: calcium, protein, DHA (if using fortified yogurt), fiber

- Salmon with quinoa and steamed vegetables (about 500–550 kcal)
  - Key nutrients: omega-3s (DHA), protein, iron, iodine (from fish), calcium (if dairy is included)

- Spinach and lentil curry with brown rice (about 450–500 kcal)
  - Key nutrients: iron, folate, fiber, protein

- Veggie omelet with cheese and a slice of whole-grain toast (about 350–420 kcal)
  - Key nutrients: protein, iron (if yolks/greens), calcium (cheese)

- Beef or turkey stir-fry with broccoli and brown rice (about 450–550 kcal)
  - Key nutrients: heme iron, protein, B vita